In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.sparse import hstack
import pickle
from tqdm import tqdm
import pickle

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)


In [3]:
physical_devices

[]

In [37]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout, Input, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
df=pd.read_csv("task-1/data/cleaned_data.csv")
y_true=df['is_duplicate']

In [6]:
transformer=pickle.load(open("task-1/data/tfidf/tfidf_transformer.pkl", "rb"))

In [7]:
question1=df["question1"][:5000]
question2=df["question2"][:5000]

In [8]:
all_questions = list(question1) + list(question2)
len(all_questions)

10000

In [9]:
vectorizer = TfidfVectorizer()
# analyze = vectorizer.build_analyzer()
transformer=vectorizer.fit(all_questions)

In [10]:
question1_vectors=transformer.transform(question1)
question2_vectors=transformer.transform(question2)

In [11]:
question1_vectors = question1_vectors.todense()
question2_vectors = question2_vectors.todense()

In [12]:
question1_vectors[5].shape

(1, 10183)

In [13]:
x=np.hstack((question1_vectors[:],question2_vectors[:]))
x.shape

(5000, 20366)

In [14]:
max(x[1])

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [15]:
x_train,x_test,y_train,y_test = train_test_split(x,y_true[:5000],test_size=0.2,random_state=40)

In [38]:
model=Sequential()
# model.add(Input(shape=(20366,)))
# model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
#model.add(Bidirectional(LSTM(128,return_sequences=True)))
# model.add(Bidirectional(LSTM(128,input_shape=(20366,))))
# model.add(Dense(128, activation = "relu"))    
# model.add(Dropout(0.5))
# model.add(Dense(1, activation = "softmax"))
model.add(GRU(256,activation="relu",input_shape=(20366,1,)))
model.add(Dense(128,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [39]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [40]:
history = model.fit(x_train, y_train, epochs = 20, batch_size = 8, validation_data= (x_test, y_test), verbose=1)

Epoch 1/20
  2/500 [..............................] - ETA: 1:46:04 - loss: 0.6919 - accuracy: 0.8125

KeyboardInterrupt: 